In [1]:
import os
import numpy as np
from os import listdir
from PIL import Image
import time
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
path="C:/DL_Dataset/Project_data"

In [3]:
def load_npdata(path):
    for r, d, f in os.walk(path):
        direc=d
        break

    np_images=[]
    
    start= time.time()
    for d in direc:
        dic={}
        sub=[a for a in listdir(path+"/"+d)]
        flag=0
        for ele in sub:
            mypath = path+"/"+d+"/"+ele+"/"
            onlyfiles = [mypath+f for f in listdir(mypath)]
            temp=[]
            for addr in onlyfiles:
                temp.append(np.array(Image.open(addr)))   
            if len(temp)>0:
                if ele[0].lower()=='f':
                    dic['father']=temp
                elif ele[0].lower()=='m':
                    dic['mother']=temp
                elif ele.lower()=='child_male':
                    dic['child']=temp    
                    dic['gender']=np.zeros((temp[0].shape))
                elif ele.lower()=='child_female':
                    dic['child']=temp    
                    dic['gender']=np.ones((temp[0].shape))    
            else:
                flag=1
                break

        if flag!=1:
            for x in dic['father']:
                for y in dic['mother']:
                    for z in dic['child']:
                        np_images.append([x,y,dic['gender'],z])
    print("time:",time.time()-start)
    return np_images

In [9]:
full_data = load_npdata(path)
print(len(full_data))

time: 45.643808126449585
1780631


In [16]:
class Autoencoder(object):

    def __init__(self, learning_rate=1e-4, batch_size=64, n_z=16 ):
        # Set hyperparameters
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.n_z = n_z #latent dimension of the encoder

        # Build the graph
        self.build()

        # Initialize paramters
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
    
    def conv2d(self,input, kernel_size, stride, num_filter):
        stride_shape = [1, stride, stride, 1]
        filter_shape = [kernel_size, kernel_size, input.get_shape()[3], num_filter]

        W = tf.get_variable('w', filter_shape, tf.float32, tf.random_normal_initializer(0.0, 0.02))
        b = tf.get_variable('b', [1, 1, 1, num_filter], initializer=tf.constant_initializer(0.0))
        return tf.nn.conv2d(input, W, stride_shape, padding='SAME') + b
    
    def de_conv2d(self,input, kernel_size, stride, output_shape,num_filter):
        stride_shape = [1, stride, stride, 1]
        filter_shape = [kernel_size, kernel_size, input.get_shape()[3], num_filter]
        W = tf.get_variable('w', filter_shape, tf.float32, tf.random_normal_initializer(0.0, 0.02))
        b = tf.get_variable('b', [1, 1, 1, num_filter], initializer=tf.constant_initializer(0.0))
        return tf.nn.conv2d_transpose(input, W,output_shape, stride_shape, padding='SAME') + b

    def max_pool(self,input, kernel_size, stride):
        ksize = [1, kernel_size, kernel_size, 1]
        strides = [1, stride, stride, 1]
        return tf.nn.max_pool(input, ksize=ksize, strides=strides, padding='SAME')

    # Build the netowrk and the loss functions
    def build(self):
        # TODOs
        #reset the graph
#         tf.reset_default_graph()
        K.clear_session()
        self.x=tf.placeholder(name='x',dtype=tf.float32,shape=[2,224,224,3])
        
        
        # Encode :f
        # x -> z
        with tf.variable_scope('conv1'):
            conv1 = self.conv2d (self.x, 128, 1, 3)
            relu1 = tf.nn.relu(conv1)
            pool1 = self.max_pool(relu1, 16, 2) #shape is (2,112,112,3)

        with tf.variable_scope('conv2'):
            conv2 = self.conv2d (pool1, 64, 1, 3)
            relu2 = tf.nn.relu(conv2)
            pool2 = self.max_pool(relu2, 8, 2) # shape is (2,56,56,3)

        with tf.variable_scope('conv3'):    
            conv3 = self.conv2d (pool2, 32, 1, 3)
            relu3 = tf.nn.relu(conv3)
            pool3 = self.max_pool(relu3, 4, 2) # shape is (2,28,28,3)
        
    
        # Decode :g
        # z -> x_hat
        # TODOs
        with tf.variable_scope('deconv1'):
            deconv1 = self.de_conv2d(pool3,32,2,(2,56,56,3),3)
        
        with tf.variable_scope('deconv2'):
            deconv2 = self.de_conv2d(deconv1,64,2,(2,112,112,3),3)
        
        with tf.variable_scope('deconv3'):
            deconv3 = self.de_conv2d(deconv2,128,2,(2,224,224,3),3)
        
        
        # Loss
        # Reconstruction loss
        # Minimize the cross-entropy loss
        # H(x, x_hat) = -\Sigma x*log(x_hat) + (1-x)*log(1-x_hat)
        
        eps=1e-10
        #loss of input wrt to reconstruction
        self.recon_loss = tf.reduce_mean(tf.nn.l2_loss(self.x - deconv3))
        
        
        #store loss in dict to debug
        self.losses = {'recon_loss':self.recon_loss}
        
        # Optimizer
        #define adam optimizer and minimize over self.recon_loss
        rmsprop=tf.train.optimizers.RMSprop(learning_rate = self.learning_rate)
        self.train_op = rmsprop.minimize(self.recon_loss)
        
        return 

    # Execute the forward and the backward pass
    def run_single_step(self, x ):
        #fetches=[self.train_op,self.losses]
        #feed_dict=self.x:x
        
        #in this we only use the loss to see if the model is trained correctly
        
        _,losses = self.sess.run([self.train_op,self.losses],feed_dict={self.x:x})
        return losses
    
    # x -> x_hat
    def reconstructor(self, x):
        # TODOs
        #after sess run 
        #first param : return fron sess.run
        #second param: feed_dict
        x_hat=self.sess.run(self.x_hat,feed_dict={self.x:x})
        
        return x_hat
    
    # x -> z
    def transformer(self, x):
        # TODOs
        z=self.sess.run(self.z,feed_dict={self.x:x})
        return z

In [17]:
def trainer(data, model_class, learning_rate=1e-4, batch_size=64, num_epoch=3, n_z=16, log_step=1):
    # Create a model    
    model= model_class(learning_rate=learning_rate,batch_size=batch_size,n_z=n_z)

    # Training loop    
    for epoch in range(num_epoch):
        start_time=time.time()
        
        #iterate over all the data
        for imgs in data:
            #get a img from tuple
            #run a forward pass
            #batch is image and label and we only need image so we index by zero
            losses= model.run_single_step(imgs)
        print(losses)
        end_time=time.time()
        print('Epoch '+str(epoch)+' '+"Training Time: "+str(end_time-start_time))
    print("DONE TRAINING")
    return model

In [18]:
data = full_data[:1]
data = np.asarray(data)
data = data[:,[0,1]]
# stacking father and mother images for encoder
for row in data:
    row = np.concatenate((row[0],row[1]),axis=0)

In [20]:
# with tf.device('/device:GPU:0'):
encode_model = trainer(data,Autoencoder)

ValueError: Passed in object of type <class 'str'>, not tf.Tensor